# Imbalance evaluation

## Imports

In [ ]:
import os
from torch import utils
import torch
import numpy as np
import pandas as pd
from sklearn import ensemble
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours
from sklearn import tree, svm
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, Normalizer
from src.utils.const import DATA_DIR, SEED
from src.utils.util_models import fix_random
from src.visualization.visualize import barplot_multiple_balancing


### Useful path to data

In [ ]:
ROOT_DIR = os.path.join(os.getcwd(), '..')
PROCESSED_DIR = os.path.join(ROOT_DIR, DATA_DIR, 'processed')

### Fix random seed

In [ ]:
fix_random(SEED)

## Start to work

In [ ]:
final = pd.read_parquet(os.path.join(PROCESSED_DIR, 'final.parquet'))

### Add rating_discrete feature

In [ ]:
final = (final
         .assign(rating_discrete=pd.cut(final.loc[:, 'rating_mean'], bins=10, labels=False))
         .astype({'rating_discrete': 'int32'})
         .drop(columns=['rating_mean']))
final.info()

### Separate train/test

In [ ]:
data = final.loc[:, final.columns != 'rating_discrete']
target = final['rating_discrete']
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.2,
                                                                    stratify=final['rating_discrete'])

## Models definition

In [ ]:
models ={'random_forest' : ensemble.RandomForestClassifier(),
        'decision_tree' :tree.DecisionTreeClassifier(),
        'GaussianNB' : GaussianNB(),
        'quadratic_discriminant' : QuadraticDiscriminantAnalysis(store_covariance=True),
        'svm': svm.SVC()
         }

Dataset creation, where all the results will be saved inside to be plotted at the end

In [ ]:
df=pd.DataFrame({'balance':[],'model_name':[],'mean_acc':[],'mean_f1':[]})

### Training function

In [ ]:
def try_sample(train_data_inside, train_target_inside,balance:str) -> None:
    # Define evaluation procedure (here we use Repeated Stratified K-Fold CV)
    global df
    cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1)
    # Evaluate model
    scoring = ['accuracy', 'f1_weighted']

    results=[]
    for elm in models.items():
        scores = cross_validate(elm[1], train_data_inside, train_target_inside, scoring=scoring, cv=cv, n_jobs=-1)
        # summarize performance
        elemento =pd.DataFrame( {'balance':[balance],'model_name':[elm[0]],'mean_acc': [np.mean(scores['test_accuracy'])],'mean_f1':[np.mean(scores['test_f1_weighted'])]})
        df = pd.concat([df,elemento])
        print(df[df['balance']==balance])


## Transformations

In [ ]:
features = [
    'year',
    'title_length',
    'runtime',
    'rating_count',
    'tag_count'
]

scaler = ColumnTransformer(
    remainder='passthrough',
    transformers=[
        ('minmax', MinMaxScaler(), features)
    ])

norm = Normalizer(norm='l2')
pipe = Pipeline(steps=[
    ('scaler', scaler)
    #,('norm', norm) #To review after checks
])

pipe.fit(train_data)
train_data_proc = pipe.transform(train_data)
test_data_proc = pipe.transform(test_data)

### balancer MLP

TODO (?)

In [ ]:
def balancer(train_target_tmp: np.ndarray) -> utils.data.WeightedRandomSampler:
    counts = np.bincount(train_target_tmp)
    if counts.any(0):
        np.seterr(divide='ignore')
        labels_weights = 1. / counts
        labels_weights[np.isinf(labels_weights)] = 0
    else:
        np.seterr(divide=None)
        labels_weights = 1. / counts
    weights = torch.tensor(labels_weights[train_target_tmp], dtype=torch.float)
    return utils.data.WeightedRandomSampler(weights, len(weights), replacement=True)

## SMOTE

In [ ]:
smt = SMOTE(k_neighbors=4)
train_data_smt_proc, train_target_smt_proc = smt.fit_resample(train_data_proc, train_target)
try_sample(train_data_smt_proc, train_target_smt_proc,'SMOTE')

## SMOTETomek

In [ ]:
smt_tom = SMOTETomek(smote=SMOTE(k_neighbors=4), tomek=TomekLinks(sampling_strategy='majority'))
train_data_smt_tom, train_target_smt_tom = smt_tom.fit_resample(train_data, train_target)
try_sample(train_data_smt_tom, train_target_smt_tom,'SMOTETomek')

## SMOTEENN

In [ ]:
smt_enn = SMOTEENN(smote=SMOTE(k_neighbors=4), enn=EditedNearestNeighbours(n_neighbors=4))
train_data_smt_enn_proc, train_target_smt_enn_proc = smt_enn.fit_resample(train_data_proc, train_target)
try_sample(train_data_smt_enn_proc, train_target_smt_enn_proc,'SMOTEENN')

## RandomOverSampler

In [ ]:
rnd_over = RandomOverSampler()
train_data_rnd_over_proc, train_target_rnd_over_proc = rnd_over.fit_resample(train_data_proc, train_target)
try_sample(train_data_rnd_over_proc, train_target_rnd_over_proc,'RandomOverSampler')

## SMOTE with threshold

#### Min Threshold

In [ ]:
bins_count = train_target.value_counts()
for i in range(len(bins_count)):
    if bins_count[i] <= 500:
        bins_count[i] = 500

bin_sizes = bins_count.to_dict()

In [ ]:
smt_new = SMOTE(k_neighbors=4, sampling_strategy=bin_sizes)
train_data_smt_new, train_target_smt_new = smt_new.fit_resample(train_data_proc, train_target)
try_sample(train_data_smt_new, train_target_smt_new,'SMOTE_threshold')

## Comparison between models

Saving the results for visualization purpose

In [ ]:
df.to_csv(os.path.join(PROCESSED_DIR, 'balancing.csv'))

In [ ]:
barplot_multiple_balancing(df, 'Comparison of balancing strategy between models')